# Clock Game Sync Word Finder

In [17]:
Numbers = [
    'dodici',
    'uno',
    'due',
    'tre',
    'quattro',
    'cinque',
    'sei',
    'sette',
    'otto',
    'nove',
    'dieci',
    'undici',
]

nNumbers = len(Numbers)

In [3]:
Automaton = [[len(n) % nNumbers, (len(n) + i) % nNumbers] for i, n in enumerate(Numbers)]
Automaton

[[6, 6],
 [3, 4],
 [3, 5],
 [3, 6],
 [7, 11],
 [6, 11],
 [3, 9],
 [5, 0],
 [4, 0],
 [4, 1],
 [5, 3],
 [6, 5]]

In [12]:
def Delta(state, move):
    return Automaton[state][move]

In [13]:
def Execute(word, starting_state):
    state = starting_state
    for c in word:
        state = Delta(state, int(c))
    return state

In [14]:
def hasKnownSyncWord(SyncWords, word):
    for w in SyncWords:
        if w in word:
            return True
    return False

In [25]:
iter = 10000
SyncWords = []

for n in range(iter):
    word = str(format(n + 2, 'b'))[1:]
    if not hasKnownSyncWord(SyncWords, word):
        outSet = set()

        for i in range(nNumbers):
            output = Execute(word, i)
            outSet.add(output)
            
        if(len(outSet) == 1):
            SyncWords.append(word)
            print(n, word.replace('0', 'A').replace('1', 'B') + ' ->', outSet)

30 AAAAA -> {3}
98 BAABAA -> {3}
252 BBBBBBA -> {6}
322 ABAAABAA -> {3}
494 BBBBAAAA -> {3}
530 AAAABABAA -> {3}
572 AAABBBBBA -> {6}
700 ABABBBBBA -> {6}
706 ABBAAABAA -> {3}
1012 BBBBBABBA -> {3}
1074 AAAABBABAA -> {3}
1134 AAABBBAAAA -> {3}
1306 ABAAABBBAA -> {3}
1309 ABAAABBBBB -> {11}
1330 ABAABBABAA -> {3}
1390 ABABBBAAAA -> {3}
1468 ABBABBBBBA -> {6}
1618 BAABABABAA -> {3}
2138 AAAABABBBAA -> {3}
2141 AAAABABBBBB -> {11}
2162 AAAABBBABAA -> {3}
2292 AAABBBBABBA -> {3}
2674 ABAABBBABAA -> {3}
2804 ABABBBBABBA -> {3}
2842 ABBAAABBBAA -> {3}
2845 ABBAAABBBBB -> {11}
2866 ABBAABBABAA -> {3}
2926 ABBABBBAAAA -> {3}
3250 BAABABBABAA -> {3}
3994 BBBBAABBBAA -> {3}
3997 BBBBAABBBBB -> {11}
4018 BBBBABBABAA -> {3}
4314 AAAABBABBBAA -> {3}
4317 AAAABBABBBBB -> {11}
5202 ABAAABABABAA -> {3}
5226 ABAAABBABBAA -> {3}
5338 ABAABBABBBAA -> {3}
5341 ABAABBABBBBB -> {11}
5362 ABAABBBBABAA -> {3}
5746 ABBAABBBABAA -> {3}
5876 ABBABBBBABBA -> {3}
6490 BAABABABBBAA -> {3}
6493 BAABABABBBBB -> {11}
